In [ ]:

"""
China's 1:1 million public version of topographic data (2021) includes threats datasets for Invest Habitat Quality, such as railroads, roads, and other infrastructure.
This script is designed to read multiple GBD files, extract specified threats layers, and rasterize them into a single GeoTIFF file.
author: Shiyu Deng
date: 2025-06-20
email:shiyu.deng.23@ucl.ac.uk
"""

import os
import fiona
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# identify the target threst layers to be extracted, take the example of roads 
folder_path = "/.../Threats_InVEST"

target_layers = ["LRDL","LFCL","LFCP"]  

gdf_list = []

# loop through all GDB files in the specified folder
for filename in os.listdir(folder_path):
    if filename.endswith(".gdb") and not filename.startswith("._"):
        gdb_path = os.path.join(folder_path, filename)
        
        try:
            
            layers = fiona.listlayers(gdb_path)
            
            # read the GDB file and extract the target layers
            for target_layer in target_layers:
                if target_layer in layers:
                    gdf = gpd.read_file(gdb_path, layer=target_layer)
                   
                    gdf_list.append(gdf)
                else:
                    print(f"File {filename} does not contain the {target_layer} layer")
        except Exception as e:
            print(f"Error reading {filename}: {e}")

if gdf_list:
    combined_gdf = pd.concat(gdf_list, ignore_index=True)

    # transform the combined GeoDataFrame to a common CRS (WGS 84)
    combined_gdf = combined_gdf.to_crs(epsg=32650)

    # indentify the bounding box of the combined GeoDataFrame
    minx, miny, maxx, maxy = combined_gdf.total_bounds
    resolution = 100  # identify the resolution of the raster, in meters
    
    width = int((maxx - minx) / resolution)
    height = int((maxy - miny) / resolution)

    transform = from_bounds(minx, miny, maxx, maxy, width, height)

    # rasterize the geometries into a raster format and assign a value of 1 to all geometries
    shapes = ((geom, 1) for geom in combined_gdf.geometry)  
    raster = rasterize(shapes, out_shape=(height, width), transform=transform, fill=0, dtype='uint8')

    # 保存为GeoTIFF格式
    with rasterio.open(
        "/.../invest/habitatquality/road_threat.tif", 'w',
        driver='GTiff',
        height=raster.shape[0],
        width=raster.shape[1],
        count=1,
        dtype=raster.dtype,
        crs=combined_gdf.crs.to_string(),  
        transform=transform
    ) as dst:
        dst.write(raster, 1)

    print("Raster file 'road.tif' has been created successfully.")
else:
    print("No valid data found.")

/var/folders/xt/zxpvbvsx2zb75gd2395zm9000000gn/T/ipykernel_62489/2556042846.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gdf = pd.concat(gdf_list, ignore_index=True)


Raster file 'railroad.tif' has been created successfully.


In [ ]:

"""
This script is designed to read multiple NetCDF files containing land cover data, extract the dominant land use type for each grid cell, clip, reproject and save the results as GeoTIFF files.
And also extract crop areas and reproject them to a
The land cover data are projected and downscaled Land Cover datasets by GCAM-Demeter model.
author: Shiyu Deng
date: 2025-06-20
email:shiyu.deng.23@ucl.ac.uk
"""

import xarray as xr
import numpy as np
import os
import geopandas as gpd
import rioxarray
from rasterio.features import geometry_mask
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap, BoundaryNorm
from rasterio.enums import Resampling


# input folder paths
folder_path = ".../demeter/outputs_fraction/ref/spatial_landcover_netcdf"  # demeter model outputs folder
shapefile_path = "/.../China_shp.shp"  # 
invest_file="/.../invest/habitatquality"

shapefile = gpd.read_file(shapefile_path)

nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]

# identify the mapping of land use types to integers
land_use_mapping = {
    0: 1,   # water
    1: 2,   # forest
    2: 3,   # shrub
    3: 4,   # grass
    4: 5,   # city
    5: 6,   # snow
    6: 7,   # sparse
    7: 8    # crops
}


land_use_labels = ['Water', 'Forest', 'Shrub', 'Grass', 'Urban', 'Snow', 'Sparse', 'Crops','NA']
colors = ['#1f78b4', '#33a02c', '#ff7f00', '#6a3d9a', '#b15928', '#e31a1c', '#a6cee3', '#fb9a99','white']
cmap = ListedColormap(colors)

# create legend elements for the land use types
legend_elements = [Patch(facecolor=colors[i], edgecolor='black', label=land_use_labels[i]) for i in range(len(land_use_labels))]

# identify the color boundaries for the discrete colormap
bounds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # 每个类别的边界
norm = BoundaryNorm(bounds, cmap.N)    # 创建离散的颜色映射

# loop through each NetCDF file
for nc_file in nc_files:
    file_path = os.path.join(folder_path, nc_file)
    
    
    ds = xr.open_dataset(file_path)
    
    # stack the land use data into a 3D array
    land_use_data = np.stack([
        ds["water"].values,  
        ds["PFT0"].values,   
        ds["PFT1"].values,   
        ds["PFT2"].values,   
        ds["PFT3"].values,   
        ds["PFT4"].values,   
        ds["PFT5"].values,   
        ds["PFT6"].values    
    ], axis=-1)
    
    # find the dominant land use type for each grid cell
    dominant_land_use_type = np.argmax(land_use_data, axis=-1)
    
    # map the dominant land use type to the corresponding integer values
    mapped_dominant_land_use = np.vectorize(land_use_mapping.get)(dominant_land_use_type)
    

    dominant_land_use_da = xr.DataArray(mapped_dominant_land_use, 
                                        coords=[ds.coords['latitude'], ds.coords['longitude']], 
                                        dims=['latitude', 'longitude'],
                                        name="Value")       

    
    dominant_land_use_da.attrs['description'] = "1: Water, 2: Forest, 3: Shrub, 4: Grass, 5: Urban, 6: Snow, 7: Sparse, 8: Crops, 255:NA"
    
# project the DataArray to WGS84 (EPSG:4326) 
    dominant_land_use_da = dominant_land_use_da.rio.write_crs("EPSG:4326")
    shapefile = shapefile.set_crs("EPSG:4326", allow_override=True)
    dominant_land_use_da = dominant_land_use_da.rio.write_nodata(255) 
        
    unique_values = np.unique(dominant_land_use_da)   
        
    # clip the DataArray to the shapefile geometry
    clipped_da = dominant_land_use_da.rio.clip(
        shapefile.geometry,
        shapefile.crs,
        drop=True,
        invert=False)
    clipped_da = clipped_da.fillna(255)   
    # set crop areas to 1 and others to 0
    crop_data = xr.where(clipped_da == 8, 1, 0)
        
    # resample and reproject the clipped cropland DataArray to EPSG:32650 with a resolution of 1000 meters, for InVEST threat
    reprojected_crop = crop_data .rio.reproject(
            "EPSG:32650",  
            resolution=1000, 
            resampling=Resampling.nearest  
        )
    reprojected_crop = reprojected_crop.fillna(255)  
    reprojected_crop_file = os.path.join(folder_path, f"reprojected_crop_{nc_file}")
    reprojected_crop.rio.to_raster(reprojected_crop_file)
    print(f"The reprojected data has been saved to: {reprojected_crop_file}")
       
    # resample and reproject the clipped Dominant Land Use DataArray to EPSG:32650 with a resolution of 1000 meters, for InVEST habitat quality
    reprojected_luc = clipped_da .rio.reproject(
            "EPSG:32650",  
            resolution=1000,  
            resampling=Resampling.nearest  
        )
    reprojected_luc = reprojected_luc.fillna(255)  
    reprojected_luc_file = os.path.join(folder_path, f"reprojected_luc_{nc_file}")
    reprojected_luc.rio.to_raster(reprojected_luc_file)
    print(f"The reprojected data has been saved to: {reprojected_luc_file}")
